
# Trend Factor Analysis

This notebook explores the trend factor in digital assets.

To conduct this analysis, we will use **FactorLab**, an open-source python package we have created specifically for alpha and risk factor analysis. 

To install **FactorLab**:
`pip install factorlab`

In [2]:
# uncomment to install factorlab
# pip install factorlab

In [7]:
import pandas as pd
import numpy as np

from functools import partial
from typing import Optional, List, Callable
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import plotly.graph_objects as go
import seaborn as sns
%matplotlib inline
from importlib import resources
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

from factorlab.trend import Trend
from factorlab.transform import Transform
from factorlab.data_viz import plot_table, plot_series, plot_bar, add_fonts
from factorlab.time_series_analysis import linear_reg, adf, hurst, granger_causality
from factorlab.factor_analysis import Factor
from factorlab.performance import Performance
from factorlab.param_grid_search import *

ModuleNotFoundError: No module named 'time_series_analysis'

# Collect Data

First, let's pull our value factor data csv files.

Data was gathered, cleaned and wrangled using **CryptoDataPy**, our open source python package. See the data wrangling notebooks for the data pre-processing code.

In [ ]:
df = pd.read_csv('../src/factorlab/datasets/data/crypto_market_data.csv', index_col=[0,1], parse_dates=True)

In [ ]:
# merge
oc_df = pd.concat([mktcap_df, oc_df], axis=1).sort_index()

In [ ]:
# addresses
add_df = oc_df[['add_act', 'add_new', 'add_tot']].copy()

- Compute an **adjusted transaction value** to reduce impact of large outlier transactions by multiplying average transfer value by transaction counts

In [11]:
oc_df['tfr_val_usd_adj'] = oc_df.tx_count * oc_df.tfr_val_mean_usd

- Compute **Network value to Transactions value NVT ratio**

In [12]:
oc_df['nvt'] = oc_df.mkt_cap / oc_df.tfr_val_usd
oc_df['nvt_adj'] = oc_df.mkt_cap / oc_df.tfr_val_usd_adj

# Feature and Target Variables

## Target Variables

We will need to compute spot, total and forward returns which will be our target variable.

We also compute the equal-weighted and market cap weighted market return to estimate betas/alphas.

In [ ]:
# total returns
ret_df = Transform(df.close).returns().rename(columns={'close':'ret'})
ret_df['funding_rate'] = df.funding_rate.fillna(0)
ret_df['tr'] = ret_df.ret - ret_df.funding_rate

In [ ]:
# fwd rets
fwd_spot_ret = Transform(df.close).returns(lags=1, forward=True).close.to_frame('fwd_ret_1')
fwd_spot_ret['fwd_ret_5'] = Transform(df.close).returns(lags=5, forward=True)
fwd_spot_ret['fwd_ret_7'] = Transform(df.close).returns(lags=7, forward=True)
fwd_spot_ret['fwd_ret_10'] = Transform(df.close).returns(lags=10, forward=True)
fwd_spot_ret['fwd_ret_14'] = Transform(df.close).returns(lags=14, forward=True)
fwd_spot_ret['fwd_ret_20'] = Transform(df.close).returns(lags=20, forward=True)
fwd_spot_ret['fwd_ret_30'] = Transform(df.close).returns(lags=30, forward=True)
fwd_spot_ret['fwd_ret_60'] = Transform(df.close).returns(lags=60, forward=True)
fwd_spot_ret['fwd_ret_90'] = Transform(df.close).returns(lags=90, forward=True)
fwd_spot_ret['fwd_ret_180'] = Transform(df.close).returns(lags=180, forward=True)
fwd_spot_ret['fwd_ret_365'] = Transform(df.close).returns(lags=365, forward=True)

In [ ]:
# total fwd rets
fwd_ret = ret_df.tr.groupby('ticker').shift(-1).fillna(0).to_frame('fwd_ret')
# normalized fwd ret
fwd_ret_norm = Transform(fwd_ret).normalize_ts(window_type='expanding')

In [ ]:
# normalized fwd spot rets
fwd_spot_ret_z = Transform(fwd_spot_ret).normalize_ts(window_type='expanding')

In [ ]:
# relative returns for cross-section 
fwd_rel_ret = fwd_spot_ret.copy()
for col in fwd_rel_ret.columns:
    fwd_rel_ret[col] = fwd_spot_ret[col] - fwd_spot_ret[col].groupby('date').mean()

In [ ]:
# normalize relative returns
fwd_rel_ret_z = Transform(fwd_rel_ret).normalize_ts(window_type='expanding')

In [ ]:
# create market cap weights and returns
mkt_df = pd.concat([df, mktcap_df], axis=1)
mkt_df['mkt_cap_weights'] = mkt_df.mkt_cap / mkt_df.mkt_cap.groupby('date').sum()
mkt_df['ret'] = Transform(mkt_df.close).returns()

In [ ]:
# create ew and mw market returns
mkt_ret_ew = mkt_df.ret.groupby('date').mean().to_frame('mkt_ret')
mkt_ret_mw = (mkt_df.mkt_cap_weights * mkt_df.ret).groupby('date').sum().to_frame('mkt_ret')

## Quality Factors

We compute various types of quality factors relying on the cryptoasset and traditional asset literature:

1. **Network size**
2. **Network growth**
3. **Network safety** 


1. **Network Size**: number of addresses

In [ ]:
net_size = Quality(add_df).network_size()

In [ ]:
net_size

2. **Network Health**: 
- *Growth rate*: address growth rate over n-lookback window.
- *Persistence*: coefficient of address growth regressed on lagged value t-1 of address growth over n-lookback window.

#### Growth

In [ ]:
net_gr7 = Quality(add_df, window_size=7).network_growth()
net_gr14 = Quality(add_df, window_size=14).network_growth()
net_gr21 = Quality(add_df, window_size=21).network_growth()
net_gr30 = Quality(add_df, window_size=30).network_growth()
net_gr60 = Quality(add_df, window_size=60).network_growth()
net_gr90 = Quality(add_df, window_size=90).network_growth()

In [ ]:
net_gr7

#### Persistence

In [ ]:
net_pers = Quality(add_df, smoothing=None, window_size=30).network_persistence()

In [ ]:
net_pers

3. **Network Safety**: 
- *Volatility*: standard deviation of network growth 
- *Market beta*: beta of asset returns regressed on market returns over n-period lookback window.
- *Network alpha*:
- *Network beta*:

#### Volatility

In [ ]:
net_vol = Quality(add_df, smoothing=None, window_size=90).network_vol()

In [ ]:
net_vol.add_act.dropna().unstack().iloc[-1].sort_values()

#### Market Beta

In [ ]:
mkt_beta = Quality(add_df, window_size=60).market_beta(ret_df.ret, mkt_ret_ew.mkt_ret)

In [ ]:
mkt_beta.beta.dropna().unstack().iloc[-1].sort_values()[:20]

# Data Exploration

We can now start with some simple exploratory analysis.

### Value factor correlations

In [ ]:
# value_z_df.corr()


Converting our data into logs, we can establish which indicators have the highest correlation with market cap.

In [ ]:
# correlations
# np.log(oc_df).corr().mkt_cap.sort_values()

# Statistical Analysis

In our statistical analysis, we'll explore the relationship between our value measures and price/forward returns.

This will include:

1. *Screening methods*
2. *Regression Analysis*


### Filter Methods

Filter-based methods are a category of supervised feature selection technique that are independent from the ML algorithm and can be used to evaluate the relationship between features (value measures) and the target variable (forward returns), and rank the features based on the strength of their statistical relationship with the target.

The `Factor` class has a `filter` method which allows us to run a filter on our value measures and forward returns. Standardized value measures are first discretized into quantiles/bins and ranked by the selected statistical measure. Here we used Spearman correlation.

#### Cross sectional

In [ ]:
# compute stats for cross sectional strategies
stats_cs_1d_df = Factor(net_gr30, fwd_rel_ret_z.fwd_ret_1, strategy='cs_ls', factor_bins=5, target_bins=3).filter(metrics='all', rank_on='spearman_r')

In [ ]:
stats_cs_1d_df

In [ ]:
# compute stats for cross sectional strategies
stats_cs_7d_df = Factor(net_gr30, fwd_rel_ret_z.fwd_ret_7, strategy='cs_ls', factor_bins=5, target_bins=3).filter(metrics=['spearman_r', 'p-val', 'cramer_v', 'chi2', 'mutual_info', 'autocorrelation'], rank_on='spearman_r')

In [ ]:
stats_cs_7d_df

In [ ]:
# compute stats for cross sectional strategies
stats_cs_30d_df = Factor(net_gr30, fwd_rel_ret_z.fwd_ret_30, strategy='cs_ls', factor_bins=5, target_bins=3).filter(metrics=['spearman_r', 'p-val', 'cramer_v', 'chi2', 'mutual_info', 'autocorrelation'], rank_on='spearman_r')

In [ ]:
stats_cs_30d_df

In [ ]:
# compute stats for cross sectional strategies
stats_cs_90d_df = Factor(net_gr30, fwd_rel_ret_z.fwd_ret_90, strategy='cs_ls', factor_bins=5, target_bins=3).filter(metrics=['spearman_r', 'p-val', 'cramer_v', 'chi2', 'mutual_info', 'autocorrelation'], rank_on='spearman_r')

In [ ]:
stats_cs_90d_df

### Regression Analysis

We can use the **FactorLab** `regression` method in the `Factor` class to run *pooled* and *Fama-Macbeth* regressions in order to assess the economic and statistical significance of our value measures for both time series and cross sectional implementations. Regressions use a robust estimator (Newey-West) to account for heteroskedasticity and autocorrelation.

Value measures are ranked by betas to forward returns. We use 1, 7, 30 and 90 day forward return windows. Both value measures and forward returns are normalized (z-score).

#### Fama Macbeth


In [ ]:
fm_1d_reg = Factor(net_gr30, fwd_spot_ret_z.fwd_ret_1).regression(method='fama-macbeth', nobs=3)

In [ ]:
fm_1d_reg

In [ ]:
fm_7d_reg = Factor(net_gr30, fwd_spot_ret.fwd_ret_7).regression(method='fama-macbeth', nobs=3)

In [ ]:
fm_7d_reg

In [ ]:
fm_30d_reg = Factor(net_gr30, fwd_spot_ret_z.fwd_ret_30).regression(method='fama-macbeth', nobs=3)

In [ ]:
fm_30d_reg

In [ ]:
fm_90d_reg = Factor(net_gr30, fwd_spot_ret_z.fwd_ret_90).regression(method='fama-macbeth', nobs=3)

In [ ]:
plot_table(fm_90d_reg, fig_size=(5,3), font='Lato', title='Fama-Macbeth Regressions', subtitle='90 day forward returns')

# Factor Return Analysis

In our factor return analysis, we'll explore the profitability of our value factors for both time series and cross-sectional trading strategies.

Our factor analysis involves computing factor returns by scaling forward returns on value factor signals (forecasts). Note that this analysis is carried out with little or no optimization since the goal is to uncover robust alpha/risk factors, rather than overfitting the data.

We will consider a few key properties of our factor returns:

- **Factor quantiles**: we evaluate forward returns over varying window lengths across factor quantiles. Robust factors-based strategies will typically show a monotonic relationship between factor quantiles and forward returns.
- **Cumulative returns**: cumulative returns, aka the equity curve, give us a good sense of whether the factor-based strategy has merit. A robust and profitable factor will produce returns that go from the lower left to upper right.
- **Performance metrics**: factor returns can be evaluated like any asset or strategy returns, using measures of risk and retun. We are looking for factor that produce high risk-adjusted returns which are distinct from those produced by long exposure to the market (aka alpha). 
- **Parameter grid search**: factor returns can be very sensitive to parameter choices, e.g. window size/lookback in momentum strategies. A parameter grid search allows us to assess how factors performn across a broad range of parameters. We are looking for a parameter grid space which is robust to small changes in parameter values.


## Factor Quantiles

The `Factor` class has a `ret_quantiles` method which allows us to compare average forward returns by factor quantile.

#### Time Series

In [ ]:
# quantile_ts_ret = Factor(value_z_df, fwd_spot_ret[['fwd_ret_1', 'fwd_ret_7', 'fwd_ret_30']], strategy='ts_ls', factor_bins=3).ret_quantiles(factor='nvm_ratio_an30', plot_rets=True)

In [ ]:
# quantile_ts_ret = Factor(value_z_df, fwd_spot_ret[['fwd_ret_1', 'fwd_ret_7', 'fwd_ret_30']], strategy='ts_ls', factor_bins=3).ret_quantiles(factor='nvm_ratio_aa30', plot_rets=True)

In [ ]:
# quantile_ts_ret = Factor(value_z_df, fwd_spot_ret[['fwd_ret_1', 'fwd_ret_7', 'fwd_ret_30']], strategy='ts_ls', factor_bins=3).ret_quantiles(factor='nvt_ratio_30', plot_rets=True)

In [ ]:
# quantile_ts_ret = Factor(value_z_df, fwd_spot_ret[['fwd_ret_1', 'fwd_ret_7', 'fwd_ret_30']], strategy='ts_ls', factor_bins=3).ret_quantiles(factor='mvrv', plot_rets=True)

#### Cross-sectional

In [ ]:
quantile_cs_ret = Factor(net_gr7, fwd_spot_ret.fwd_ret_1, strategy='cs_ls', factor_bins=5).quantiles(factor='add_act', metric='ann_ret', rebalancing=14)

In [ ]:
plot_bar(quantile_cs_ret, title='Sorted Portfolio Returns by Factor Quantile')

In [ ]:
quantile_cs_ret = Factor(net_gr7, fwd_spot_ret.fwd_ret_1, strategy='cs_ls', factor_bins=5).quantiles(factor='add_new', metric='ann_ret', rebalancing=14)

In [ ]:
plot_bar(quantile_cs_ret, title='Sorted Portfolio Returns by Factor Quantile')

In [ ]:
quantile_cs_ret = Factor(net_gr7, fwd_spot_ret.fwd_ret_1, strategy='cs_ls', factor_bins=5).quantiles(factor='add_tot', metric='ann_ret', rebalancing=14)

In [ ]:
plot_bar(quantile_cs_ret, 
         title='Quantile Returns', 
         subtitle='Annual return by factor quantile',
         y_label='annual return',
         add_line=True,
         font='Lato',
         source='Strategy Risks'
        )

## Cumulative Returns

#### Time Series Strategy

Time strategies go long assets with high factor values, and short assets with low factor values relative to their past history. Factor values are converted to signals on which one-period ahead forward returns are scaled to obtain factor returns.

Strategy parameters are not optimized and are set to:

- `signal_type`: value factors are converted to continuous signals between 1 and -1.
- `rebalancing`: weekly/7 day portfolio rebalancing
- `t_cost`: 20 bps transaction cost
- `weighting`: signals are adjusted by a volatility scaling factor that targets 10% volatility.

The `Factor` class has a `returns` method which allows us to compute factor returns with specific strategy parameters.

In [ ]:
# ts carry returns
# value_ts_ret = Factor(value_df, ret_df.tr, strategy='ts_ls', factor_bins=3).returns(signal_type='signal', rebalancing=7, t_cost=0.002, weighting='iv')

The `Performance` class has a `plot_metric` method which plots a time series of any financial performance metric. Here we plot cumulative returns.

In [ ]:
# Performance(value_ts_ret.loc['2016-01-01':,:], mkt_ret=mkt_ret_mw, ret_type='log').plot_metric(metric='cumulative_ret')

#### Cross-Sectional Strategy
Cross-sectional strategies sort assets into quantiles based on their factor values. A long portfolio is formed from the top quantile, and a short portfolio from the bottom quantile.

Strategy parameters are not optimized and are set to:
- `quantiles`: 5
- `tails`: two (use only top and bottom quantiles)
- `cs_norm`: normalize value measures over time series before creating quantiles.
- `rebalancing`: weekly/7 day portfolio rebalancing
- `t_cost`: 20 bps transaction cost
- `weighting`: inverse vol (iv)

In [ ]:
# cs returns
value_cs_ret = Factor(net_vol, ret_df.tr, strategy='cs_ls', factor_bins=5).returns(signal_type='signal_quantiles', tails='two', rebalancing=7, t_cost=0.002, weighting='ew')

In [ ]:
plot_series(value_cs_ret.cumsum(), fig_size=(15,7), title='Cumulative Returns', subtitle='Long/Short Quality Strategies')

## Performance Metrics

The `Performance` class has a `table` method which computes a wide variety of financial performance metrics and ranks them on a specified metric. We use Sharpe ratio below.

In [ ]:
# perf_ts_table = Performance(value_ts_ret.loc['2016-01-01':,:], mkt_ret=mkt_ret_mw, ret_type='log').table(metrics='key_metrics', rank_on='Sharpe ratio')

In [ ]:
# publish_table(perf_ts_table, title='Performance - Time Series Value', reset_index=True)

In [ ]:
perf_cs_table = Performance(value_cs_ret, mkt_ret=mkt_ret_mw, ret_type='log').table(metrics='key_metrics', rank_on='Sharpe ratio')

In [ ]:
plot_table(perf_cs_table, fig_size=(10,2.5), title='Performance Metrics', subtitle='Cross-sectional Quality')

## Parameter Grid Search

Rather than pick an arbitrary parameter value as most academic studies do, **FactorLab** has a `factor_param_grid_search` function allows us to explore the parameter space.

A robust value factor should be insensitive to changes in inputs and small changes in parameter values. A factor which produces positive returns only across a narrow set of parameter values is more likely to be spurious.

In [ ]:
# parameter grid search
nvm_ts_param_df = factor_param_grid_search(oc_df, fwd_ret, Value, 'nvm',
                                           
                         feat_args = {
                             'log': [True, False],
                             'smoothing': ['smw','ewm','median'],
                             'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                             'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_new', 'add_act'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'ts_ls'},
                     ret_args = {'signal_type': 'signal', 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
param_heatmap(nvm_ts_param_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
param_heatmap(nvm_ts_param_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'lin_reg', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
param_heatmap(nvm_ts_param_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_act', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
nvm_ts_param_h1_df = factor_param_grid_search(oc_df.loc['2016-01-01':'2019-12-31'], fwd_ret, Value, 'nvm',
                                feat_args = {
                                    'log': [True, False],
                                  'smoothing': ['smw','ewm','median'],
                                   'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                                    'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_act', 'add_new'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'ts_ls'},
                     ret_args = {'signal_type': 'signal', 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
nvm_ts_param_h2_df = factor_param_grid_search(oc_df.loc['2020-01-01':], fwd_ret, Value, 'nvm',
                                feat_args = {
                                    'log': [True, False],
                                  'smoothing': ['smw','ewm','median'],
                                   'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                                    'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_act', 'add_new'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'ts_ls'},
                     ret_args = {'signal_type': 'signal', 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
param_heatmap(nvm_ts_param_h1_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
param_heatmap(nvm_ts_param_h2_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
nvm_cs_param_df = factor_param_grid_search(oc_df, fwd_ret, Value, 'nvm',
                                feat_args = {
                                    'log': [True, False],
                                  'smoothing': ['smw','ewm','median'],
                                   'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                                    'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_new', 'add_act'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'cs_ls', 'factor_bins':5},
                     ret_args = {'signal_type': 'signal_quantiles', 'tails': 'two', 'cs_norm': True, 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
param_heatmap(nvm_cs_param_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
param_heatmap(nvm_cs_param_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'lin_reg', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
nvm_cs_param_h1_df = factor_param_grid_search(oc_df.loc['2016-01-01':'2019-12-31'], fwd_ret, Value, 'nvm',
                                feat_args = {
                                    'log': [True, False],
                                  'smoothing': ['smw','ewm','median'],
                                   'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                                    'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_act', 'add_new'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'cs_ls', 'factor_bins': 5},
                     ret_args = {'signal_type': 'signal_quantiles', 'tails': 'two', 'cs_norm': True, 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
nvm_cs_param_h2_df = factor_param_grid_search(oc_df.loc['2020-01-01':], fwd_ret, Value, 'nvm',
                                feat_args = {
                                    'log': [True, False],
                                  'smoothing': ['smw','ewm','median'],
                                   'window_size': [3, 5, 7, 14, 21, 30, 60, 90, 120, 180, 365],
                                    'method': ['ratio', 'lin_reg']
                                },
                     algo_args = { 'act_users': ['add_act', 'add_new'],
                                  'norm': [True, False],
                                 },
                     factor_args = {'strategy': 'cs_ls', 'factor_bins': 5},
                     ret_args = {'signal_type': 'signal_quantiles', 'tails': 'two', 'cs_norm': True, 'rebalancing': 7, 't_cost': 0.002, 'weighting': 'iv'}
                               )

In [ ]:
param_heatmap(nvm_cs_param_h1_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

In [ ]:
param_heatmap(nvm_cs_param_h2_df, metric='sharpe_ratio', fixed_params={'log': True, 'method': 'ratio', 'act_users': 'add_new', 'norm': True}, plot_params=['smoothing', 'window_size'])

### T-cost Analysis

Assessing the investability of a factor involves making sure that returns are positive after transaction costs. We can estimate the breakeven transaction cost for any factor using the `tcosts_be` method.

An investable value factor should produce positive returns after transaction costs with enough of a margin should t-costs increase.

In [ ]:
be_ts_tcosts = Factor(value_df, ret_df.tr, strategy='ts_ls', factor_bins=5).tcosts_be(signal_type='signal', rebalancing=7, weighting='iv', plot_tcosts=True)

In [ ]:
be_cs_tcosts = Factor(value_df, ret_df.tr, strategy='cs_ls', factor_bins=5).tcosts_be(signal_type='signal_quantiles', cs_norm=True, tails='two', rebalancing=7, weighting='iv', plot_tcosts=True)

# Value Everywhere

We can compare the cryptoasset value factor returns to those of individual equities (US, UK, EU, JP), global equity indexes, FX, global fixed-income and commodity value factors using AQR's Value & Momentum Everywhere returns:
<br>
https://www.aqr.com/Insights/Datasets/Value-and-Momentum-Everywhere-Factors-Monthly

In [ ]:
aqr_df = pd.read_excel('https://www.aqr.com/-/media/AQR/Documents/Insights/Data-Sets/Value-and-Momentum-Everywhere-Portfolios-Monthly.xlsx', sheet_name='VME Portfolios', header=20, index_col=0, parse_dates=True)
aqr_df.index.name = 'date'

In [ ]:
# create L/S value portfolios
aqr_df['US_Eqty_Value_LS'] = aqr_df.loc[:,'VAL3US'] - aqr_df.loc[:,'VAL1US']
aqr_df['UK_Eqty_Value_LS'] = aqr_df.loc[:,'VAL3UK'] - aqr_df.loc[:,'VAL1UK']
aqr_df['EU_Eqty_Value_LS'] = aqr_df.loc[:,'VAL3EU'] - aqr_df.loc[:,'VAL1EU']
aqr_df['JP_Eqty_Value_LS'] = aqr_df.loc[:,'VAL3JP'] - aqr_df.loc[:,'VAL1JP']
aqr_df['EQ_Value_LS'] = aqr_df.loc[:,'VAL3_VME_EQ'] - aqr_df.loc[:,'VAL1_VME_EQ']
aqr_df['FX_Value_LS'] = aqr_df.loc[:,'VAL3_VME_FX'] - aqr_df.loc[:,'VAL1_VME_FX']
aqr_df['FI_Value_LS'] = aqr_df.loc[:,'VAL3_VME_FI'] - aqr_df.loc[:,'VAL1_VME_FI']
aqr_df['COM_Value_LS'] = aqr_df.loc[:,'VAL3_VME_COM'] - aqr_df.loc[:,'VAL1_VME_COM']

In [ ]:
value_ts_ret.columns

In [ ]:
# add crypto value factor
aqr_df['Crypto_Value_TS_LS'] = value_ts_ret[['nvm_ratio_aa30', 'nvm_ratio_an30', 'nvt_ratio_30', 
                                             'nvt_ratio_adj30', 'nvt_ratio_cnt30']].mean(axis=1).resample('M').sum()
aqr_df['Crypto_Value_CS_LS'] = value_cs_ret[['nvm_ratio_aa30', 'nvm_ratio_an30', 'nvt_ratio_30', 'nvt_ratio_adj30', 
                                             'nvt_ratio_cnt30']].mean(axis=1).resample('M').sum()
aqr_df['mkt_ret'] = mkt_ret_mw.resample('M').sum()

In [ ]:
# target vol
aqr_norm_df = Transform(aqr_df).target_vol(ann_vol=0.1, ann_factor=12)

In [ ]:
Performance(aqr_norm_df.loc[:,'US_Eqty_Value_LS':'Crypto_Value_CS_LS'].dropna(), mkt_ret=aqr_df.mkt_ret, ret_type='log').plot_metric()

In [ ]:
value_table = Performance(aqr_norm_df.loc[:,'US_Eqty_Value_LS':'Crypto_Value_CS_LS'], mkt_ret=aqr_norm_df.mkt_ret, ann_factor=12).table(rank_on='Sharpe ratio', metrics='key_metrics')

In [ ]:
value_table

In [ ]:
publish_table(value_table, title='Performance - Value Everywhere', reset_index=True)

In [ ]:
corr_df = aqr_df.loc[:,'US_Eqty_Value_LS':].corr()

In [ ]:
# create correlation heatmap
# set plot style, font and colors
plt.style.use('seaborn')
plt.rcParams['font.family'] = 'georgia'
fig = plt.figure(figsize=(15,15))
sns.despine(left=True)

heatmap = sns.heatmap(corr_df, cmap="vlag_r", annot=True, vmin=-1, vmax=1, xticklabels=corr_df.columns, yticklabels=corr_df.columns, square=True)
heatmap.set_title('Correlation Heatmap - Value Everywhere', fontdict={'fontsize':15}, pad=20, weight='bold')
bottom, top = heatmap.get_ylim()
heatmap.set_ylim(bottom + 0.5, top - 0.5);

# Market Valuation

In [ ]:
# market ret and NVM ratio
mkt_value_df = value_z_df[['nvm_ratio_an30']].mean(axis=1).groupby('date').mean().to_frame('NVM_ratio')
mkt_value_df = pd.concat([mkt_value_df, mkt_ret_mw.cumsum()], axis=1).replace(0, np.nan).dropna()

In [ ]:
# plot market ret and NVM ratio
fig, axes = plt.subplots(nrows=2, figsize=(15,10))

mkt_value_df['mkt_ret'].plot(
    linewidth=1,
    color='#006BA2',
    ax=axes[0])

mkt_value_df['NVM_ratio'].plot(
    kind='bar', 
    ax=axes[1], 
    width=2,
    color='#DB444B')

# font
plt.rcParams['font.family'] = 'georgia'

# grid
for i in range(0,2):
    axes[i].grid(which="major", axis='y', color='#758D99', alpha=0.6, zorder=1)
    axes[i].set_facecolor("whitesmoke")
    axes[i].spines[['top', 'right', 'left']].set_visible(False)
    
# Reformat y-axis tick labels
axes[0].set_ylabel('Cumulative returns')
axes[0].yaxis.tick_right()
axes[1].set_ylabel('NVM Ratio (z-score)')
axes[1].yaxis.tick_right()

# add systamental logo
img = Image.open('../src/factorlab/systamental_logo.png')
plt.figimage(img, origin='upper')

# Add in title and subtitle
axes[0].text(x=0.13, y=.92, s=f"Cryptoassets are no longer expensive relative to network fundamentals", transform=fig.transFigure, ha='left', fontsize=14,
        weight='bold', alpha=.8, fontdict=None)

sub_title = f"Crypto market portfolio (market-cap weighted) vs. NVM ratio (new addresses)"
axes[0].text(x=0.13, y=.89, s=sub_title, transform=fig.transFigure, ha='left', fontsize=12, alpha=.8,
        fontdict=None);

# show every Nth label
locs, labels = plt.xticks()
N = 365
plt.xticks(locs[::N], mkt_value_df.index[::N].strftime('%Y'), rotation=0)

# add over/undervaluation text
axes[1].text(0.95, .8, 'Undervalued', horizontalalignment='right', verticalalignment='bottom', transform=axes[1].transAxes)
axes[1].text(0.95, .05, 'Overvalued', horizontalalignment='right', verticalalignment='bottom', transform=axes[1].transAxes);

In [ ]:
# market value comp
value_comp_z_df = value_z_df[['nvm_ratio_an30']].mean(axis=1)
mkt_value_comp_z_df = value_comp_z_df.groupby('date').mean().to_frame('NVM_ratio')
mkt_value_comp_z_df['ticker'] = 'crypto_mkt'
mkt_value_comp_z_df = mkt_value_comp_z_df.reset_index().set_index(['date', 'ticker'])

In [ ]:
# fwd mkt rets
fwd_mkt_ret = fwd_spot_ret.groupby('date').mean()
fwd_mkt_ret['ticker'] = 'crypto_mkt'
fwd_mkt_ret = fwd_mkt_ret.reset_index().set_index(['date', 'ticker'])

In [ ]:
quantile_ts_ret = Factor(mkt_value_comp_z_df, fwd_mkt_ret[['fwd_ret_1', 'fwd_ret_7', 'fwd_ret_30', 'fwd_ret_90', 'fwd_ret_180']], strategy='ts_ls').ret_quantiles(factor='NVM_ratio', plot_rets=True)

# Crypto Valuation Dashboard

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

value_comp_z_df.unstack().iloc[-1].dropna().sort_values().plot(kind='barh', figsize=(15,10), color='#5DA4DF')

# font
plt.rcParams['font.family'] = 'georgia'

# grid
ax.grid(which="major", axis='x', color='#758D99', alpha=0.6, zorder=1)
ax.set_facecolor("whitesmoke")
ax.spines[['top', 'right', 'left']].set_visible(False)
    
# Reformat y-axis tick labels
ax.set_ylabel('NVM Ratio (z-score)')
ax.yaxis.tick_right()

# add systamental logo
img = Image.open('../src/factorlab/systamental_logo.png')
plt.figimage(img, origin='upper')

# Add in title and subtitle
ax.text(x=0.13, y=.92, s=f"Cryptoassets Valuations", transform=fig.transFigure, ha='left', fontsize=14,
        weight='bold', alpha=.8, fontdict=None)

sub_title = f"NVM ratio (new addreses))"
ax.text(x=0.13, y=.89, s=sub_title, transform=fig.transFigure, ha='left', fontsize=12, alpha=.8,
        fontdict=None);